In [41]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pathlib

In [42]:
# Find path to state csv
file = pathlib.Path("resources/states.csv")

In [44]:
# Read into state csv and convert abbrev into list
states_df = pd.read_csv(file)
state_abbrev = states_df['Code'].values.tolist()
state_abbrev


['AL',
 'AK',
 'AZ',
 'AR',
 'CA',
 'CO',
 'CT',
 'DE',
 'DC',
 'FL',
 'GA',
 'HI',
 'ID',
 'IL',
 'IN',
 'IA',
 'KS',
 'KY',
 'LA',
 'ME',
 'MD',
 'MA',
 'MI',
 'MN',
 'MS',
 'MO',
 'MT',
 'NE',
 'NV',
 'NH',
 'NJ',
 'NM',
 'NY',
 'NC',
 'ND',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VT',
 'VA',
 'WA',
 'WV',
 'WI',
 'WY']

In [36]:
# Mac Users
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

In [45]:
# Open Chrome browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [38]:
# Windows Users
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [46]:
# Create empty list to append into
indicator_list = []

# Loop through each state
for state in state_abbrev:

    # Set URL to scrape
    url = 'https://elections.mit.edu/#state-' + state
    browser.visit(url)
    
    # Add time delay
    time.sleep(3)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    # Retrieve row 
    section = soup.find('section', id='statebyYear')
    rows = section.find_all('tr', class_='indicatorRow')
    
    # Loop through each row to pull indicator and value elements
    for row in rows:
        indicator_dict = {}
        indicator_dict['state']  = state
        indicator_dict['indicator_name'] = row.find('td', class_='indicatorName').text
        indicator_dict['indicator_value'] = row.find('span', class_='valueLbl').text
        indicator_list.append(indicator_dict)
        
# Close the browser after scraping
browser.quit()


KeyboardInterrupt: 

In [40]:
print(indicator_list)

[{'state': 'AK', 'indicator_name': 'Data Completeness', 'indicator_value': '100.00%'}, {'state': 'AK', 'indicator_name': 'Disability- or Illness-related Voting Problems', 'indicator_value': '12.19%'}, {'state': 'AK', 'indicator_name': 'Mail Ballots Rejected', 'indicator_value': '0.27%'}, {'state': 'AK', 'indicator_name': 'Mail Ballots Unreturned', 'indicator_value': '13.17%'}, {'state': 'AK', 'indicator_name': 'Military and Overseas Ballots Rejected', 'indicator_value': '7.93%'}, {'state': 'AK', 'indicator_name': 'Military and Overseas Ballots Unreturned', 'indicator_value': '12.36%'}, {'state': 'AK', 'indicator_name': 'Online Registration Available', 'indicator_value': 'Yes'}, {'state': 'AK', 'indicator_name': 'Postelection Audit Required', 'indicator_value': 'Yes'}, {'state': 'AK', 'indicator_name': 'Provisional Ballots Cast', 'indicator_value': '6.13%'}, {'state': 'AK', 'indicator_name': 'Provisional Ballots Rejected', 'indicator_value': '0.08%'}, {'state': 'AK', 'indicator_name': '